<a href="https://colab.research.google.com/github/hr1588/dacon/blob/main/%EC%98%A4%EB%8A%98%EC%9D%98_%ED%8C%8C%EC%9D%B4%EC%8D%AC%20-%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Optimization 설치 및 로드

In [1]:
pip install bayesian-optimization --q

In [8]:
from bayes_opt import BayesianOptimization

- Bayesian Optimization은 하이퍼 파라미터 튜닝과 관련
- 흔히 알려진 하이퍼 파라미터 튜닝은 Grid Search, Random Search
- 위의 두 가지는 최적의 값을 찾아갈 수 없다는 문제점 발생
- Bayesian Optimization으로 문제를 해결할 수 있음

# 하이퍼 파라미터의 3가지 튜닝 방법 비교

## 1. Grid Search

- 방법 : Grid Search는 사전에 탐색할 값들을 미리 지정해주고, 그 값들의 모든 조합을 바탕으로 성능의 최고점 탐색
- 장점 : 원하는 범위를 정확하게 비교 분석
- 단점 : 시간이 오래걸림, 성능의 최고점이 아닐 가능성이 높음

- 최적화 검색(여러 개를 비교 분석해서 최고를 찾아내는 기법)이지, 최적화 탐색(성능이 가장 높은 점으로 점차 찾아가는 기법)이 아님

## 2. Random Search

- 방법 : 사전에 탐색할 값들의 범위를 지정해주고, 그 범위 속에서 가능한 조합을 바탕으로 최고점 탐색
- 장점 : Grid Search에 비해 시간이 짧게 걸리고, Grid Search보다 랜덤하게 점을 찍어 성능이 더 좋은 점으로 갈 가능성이 높음
- 단점 : 반대로 성능이 Grid Search보다 낮을 수 있고, 하이퍼 파라미터의 범위가 너무 넓으면 일반화된 결과가 나오지 않음
- seed를 고정하지 않으면 할 때마다 결과가 달라지고, Grid Search와 마찬가지로 최적화 검색의 개념


## 3. Bayeisan Optimization

- 방법 : 하이퍼파라미터의 범위를 지정한 후, Random하게 R 번 탐색한 후, B번 만큼 최적의 값을 찾아간다.
- 장점 : 정말 최적의 값을 찾아갈 수 있고, 상대적으로 시간이 덜 걸리며 결과값을 신뢰할 수 있음
- 단점 : 랜덤하게 찍은 값이 달라질 경우 최적화 하는데 오랜 시간 소요, 랜덤하게 찍은 값이 부족하면 최적의 값을 탐색하는게 불가능, 랜덤하게 찍은 값이 너무 많으면 최적화 이전에 이미 최적값을 가지고 있을 수 있음
- Bayesian Optimization은 수동적으로 하이퍼 파라미터를 튜닝하는데 좋은 결과를 가져온다.

# Bayeisan Optimization 활용

In [ ]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

In [2]:
# 라이브러리 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [22]:
# 어제 진행한 코드 깔끔하게 요약

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type','index'])

test.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,x0_red,x0_white
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0.429752,0.0,1.0
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0.785124,1.0,0.0
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0.223140,0.0,1.0
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0.280992,0.0,1.0
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0.247934,0.0,1.0


In [4]:
x = train.drop(columns = ['index','quality']) # 학습할 데이터
y = train['quality'] # 목표 변수

In [5]:
# 랜덤 포레스트의 하이퍼 파라미터 범위를 딕셔너리 형태로 저장
# key는 파라미터 이름, value는 탐색할 범위

rf_parameter_bounds = {
                      'max_depth' : (1,10), # 나무의 깊이
                      'n_estimators' : (20,100),
                      }

**n_estimators**	
- 결정트리의 갯수를 지정
- Default = 10
- 무작정 트리 갯수를 늘리면 성능 좋아지는 것 대비 시간이 걸릴 수 있음

**min_samples_split**	
- 노드를 분할하기 위한 최소한의 샘플 데이터수
→ 과적합을 제어하는데 사용
- Default = 2 → 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가

**min_samples_leaf**
- 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수
- min_samples_split과 함께 과적합 제어 용도
- 불균형 데이터의 경우 특정 클래스의 데이터가 극도로 작을 수 있으므로 작게 설정 필요

**max_features**	
- 최적의 분할을 위해 고려할 최대 feature 개수
- Default = 'auto' (결정트리에서는 default가 none이었음)
- int형으로 지정 →피처 갯수 / float형으로 지정 →비중
- sqrt 또는 auto : 전체 피처 중 √(피처개수) 만큼 선정
- log : 전체 피처 중 log2(전체 피처 개수) 만큼 선정

**max_depth**
- 트리의 최대 깊이
- default = None
→ 완벽하게 클래스 값이 결정될 때 까지 분할
또는 데이터 개수가 min_samples_split보다 작아질 때까지 분할
- 깊이가 깊어지면 과적합될 수 있으므로 적절히 제어 필요

**max_leaf_nodes**
- 리프노드의 최대 개수

## 함수 생성

In [12]:
def rf_bo(max_depth, n_estimators):
  rf_params = {
              'max_depth' : int(round(max_depth)),
               'n_estimators' : int(round(n_estimators)),      
              }
  rf = RandomForestClassifier(**rf_params)

  x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size = 0.2, )

  rf.fit(x_train,y_train)
  score = accuracy_score(y_valid, rf.predict(x_valid))
  return score

In [9]:
BO_rf = BayesianOptimization(f = rf_bo, pbounds = rf_parameter_bounds,random_state = 1004)

In [13]:
BO_rf.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.4564   |  1.351    |  88.52    |
|  2        |  0.5736   |  7.719    |  89.11    |
|  3        |  0.5636   |  6.04     |  26.44    |
|  4        |  0.5409   |  3.085    |  39.38    |
|  5        |  0.58     |  6.54     |  68.57    |
|  6        |  0.5436   |  3.077    |  91.86    |
|  7        |  0.52     |  1.684    |  77.75    |
|  8        |  0.5227   |  3.093    |  39.37    |
|  9        |  0.5355   |  3.07     |  88.64    |
|  10       |  0.5927   |  7.735    |  89.06    |


In [14]:
max_params = BO_rf.max['params']

max_params['max_depth'] = int(max_params['max_depth'])
max_params['n_estimators'] = int(max_params['n_estimators'])
print(max_params)

{'max_depth': 7, 'n_estimators': 89}
